In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from ultralytics import YOLO
import albumentations as A
from albumentations.pytorch import ToTensorV2

import os

import cv2

import matplotlib.pyplot as plt

import plotly.graph_objects as go
import plotly.express as px

from ipywidgets import widgets, VBox, HBox

from IPython.display import display, clear_output

from PIL import Image, ImageDraw, ImageFont

import base64
from io import BytesIO

In [2]:
train_df = pd.read_csv('data/train_df.csv')

In [3]:
train_df.head(3)

,pothole_id,red_point_1_x,red_point_1_y,red_point_2_x,red_point_2_y
0,p101,21.0,233.0,286.0,214.0
1,p102,297.0,176.0,297.0,352.0
2,p106,341.0,89.0,228.0,312.0


## Loading Model

In [3]:
pothole_model = YOLO('YOLO/Pothole ID/best_model.pt')

In [ ]:
input_dir = '../../../data/Patch Perfect Data/train_images'
output_dir = 'data/pothole_images'

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

In [ ]:
for index, row in train_df.iterrows():
    pothole_id = row['pothole_id']
    image_path = os.path.join(input_dir, f'{pothole_id}.jpg')
    
    # Load the image
    image = cv2.imread(image_path)
    
    # Detect pothole bounding boxes using the model
    results = pothole_model.predict(source=image, save=False, verbose=False)
    
    if len(results[0].boxes) > 0:
        # Get the bounding box with the highest confidence
        pothole_boxes = results[0].boxes.xyxy.cpu().numpy()
        pothole_confidences = results[0].boxes.conf.cpu().numpy()
        max_conf_idx = np.argmax(pothole_confidences)
        pothole_bbox = pothole_boxes[max_conf_idx].astype(int)
        
        # Extract the pothole region
        x1, y1, x2, y2 = pothole_bbox
        pothole_image = image[y1:y2, x1:x2]
    else:
        # If no pothole is detected, use the whole image
        pothole_image = image
    
    # Save the extracted pothole image
    output_path = os.path.join(output_dir, f'{pothole_id}.jpg')
    cv2.imwrite(output_path, pothole_image)

print("Pothole images extracted and saved successfully.")

## Storing Original Images

In [5]:
import os
import cv2

# Directories
input_dir = '../../../data/Patch Perfect Data/train_images'  # Directory containing the original images
output_dir = 'data/train_images'  # Directory to save the full images

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Iterate through the DataFrame and save the full images
for index, row in train_df.iterrows():
    pothole_id = row['pothole_id']
    image_path = os.path.join(input_dir, f'{pothole_id}.jpg')
    
    # Load the image
    image = cv2.imread(image_path)
    
    # Save the full image to the output directory
    output_path = os.path.join(output_dir, f'{pothole_id}.jpg')
    cv2.imwrite(output_path, image)

print("Full images saved successfully in 'train_images' directory.")

Full images saved successfully in 'train_images' directory.


## Creating Cutout images of Segmentation/images

In [7]:
input_dir = '../../../data/Patch Perfect Data/train_images'
output_dir = 'YOLO/Segmentation/pothole_cutouts/images'

In [8]:
os.makedirs(output_dir, exist_ok=True)

In [9]:
for image_name in os.listdir(input_dir):
    if image_name.endswith(('.jpg', '.jpeg', '.png')):
        image_path = os.path.join(input_dir, image_name)
        image = cv2.imread(image_path)

        # Detect pothole bounding boxes using the model
        results = pothole_model.predict(source=image, save=False, verbose=False)

        if len(results[0].boxes) > 0:
            # Get the bounding box with the highest confidence
            pothole_boxes = results[0].boxes.xyxy.cpu().numpy()
            pothole_confidences = results[0].boxes.conf.cpu().numpy()
            max_conf_idx = np.argmax(pothole_confidences)
            pothole_bbox = pothole_boxes[max_conf_idx].astype(int)
            
            # Extract the pothole region
            x1, y1, x2, y2 = pothole_bbox
            pothole_image = image[y1:y2, x1:x2]
            
            # Save the extracted pothole image
            output_path = os.path.join(output_dir, image_name)
            cv2.imwrite(output_path, pothole_image)
        else:
            # If no pothole is detected, print a message and skip saving
            print(f"No pothole detected for image {image_name}")

print("Pothole cutouts processing complete.")

No pothole detected for image p1153.jpg
No pothole detected for image p1334.jpg
No pothole detected for image p1333.jpg
No pothole detected for image p1177.jpg
No pothole detected for image p409.jpg
No pothole detected for image p1148.jpg
No pothole detected for image p1159.jpg
No pothole detected for image p1165.jpg
Pothole cutouts processing complete.
